# Multi-Region IoT - Setup

After you have launched the CloudFormation stacks in the master and slave region you will setup your environment:

* Enable [IoT registry events](https://docs.aws.amazon.com/iot/latest/developerguide/registry-events.html) in the master region
* Create a global DyanmoDB table which spans the master and slave region

## Libraries

In [ ]:
import boto3
import json
import time
import urllib.request

from os import makedirs
from shutil import copy
from time import time, gmtime, localtime, strftime

## Global Vars

In [ ]:
%store -r config

print("config: {}".format(json.dumps(config, indent=4, default=str)))

## Amazon Root CA

Get the Amazon Root CA which signed the certificate for IoT Core's MQTT message broker. The CA will be used when connecting a device to AWS IoT Core.

In [ ]:
r = urllib.request.urlopen('https://www.amazontrust.com/repository/AmazonRootCA1.pem')
cert_pem = r.read().decode()
print(cert_pem)

f = open('AmazonRootCA1.pem','w')
f.write(cert_pem)
f.close()

## Boto3 Clients
Create iot boto3 client for the master region.

In [ ]:
c_iot_m = boto3.client('iot', region_name = config['aws_region_master'])

## Enable IoT registry events

In [ ]:
response = c_iot_m.update_event_configurations(
    eventConfigurations={
        "THING_TYPE": {
            "Enabled": True
        }, 
        "JOB_EXECUTION": {
            "Enabled": True
        }, 
        "THING_GROUP_HIERARCHY": {
            "Enabled": True
        }, 
        "CERTIFICATE": {
            "Enabled": True
        }, 
        "THING_TYPE_ASSOCIATION": {
            "Enabled": True
        }, 
        "THING_GROUP_MEMBERSHIP": {
            "Enabled": True
        }, 
        "CA_CERTIFICATE": {
            "Enabled": True
        }, 
        "THING": {
            "Enabled": True
        }, 
        "JOB": {
            "Enabled": True
        }, 
        "POLICY": {
            "Enabled": True
        }, 
        "THING_GROUP": {
            "Enabled": True
        }
    }
)

print("response: {}".format(json.dumps(response, indent=4, default=str)))

## Verify
Verify that enabling IoT registry events was successfull.

In [ ]:
response = c_iot_m.describe_event_configurations()
print("response: {}".format(json.dumps(response, indent=4, default=str)))

## Create Dynamo DB global table
The global table consists of a table in the master region and one in the slave region.

Create a dynamodb boto3 client for the master region.

In [ ]:
c_dynamo_m = boto3.client('dynamodb', region_name = config['aws_region_master'])

Create the global table.

In [ ]:
response = c_dynamo_m.create_global_table(
    GlobalTableName=config['dynamo_provisioning_table'],
    ReplicationGroup=[
        {'RegionName': config['aws_region_master']},
        {'RegionName': config['aws_region_slave']}
    ]
)

print("response: {}".format(json.dumps(response, indent=4, default=str)))

### Verify
Verify that the global table has been created.

In [ ]:
response = c_dynamo_m.describe_global_table(
    GlobalTableName=config['dynamo_provisioning_table']
)

print("response: {}".format(json.dumps(response, indent=4, default=str)))